## NASA Center Analysis
<details>
<summary>IMPORTANT (FEB 2025)</summary>
<br>
NEX GDDP currently has a bug that is affected tmin (minimum temperature) and tas (average temperature) for some Centers, only for SSP 1-2.6 and 3-7.0. These are the following Centers that are affected:

AMES
LARC
GISS
JPL
JSC
KSC
WFF

That means any variable that includes tmin or tas, for SSP 1-2.6 and 3-7.0, will be affected, and we will plan on not using those variables for now. The variables include:
Max DTR, Tmin >20C, Tmin < 0C, Coldest Tmin of the Year, Annual Average Tmin, any of the humid heat diagnostics (e.g., heat index, WBGT, etc).
</details>

<details>
<summary>VARIABLES</summary>

| Variable Name       | Long Name                                          | Variable Category | Units     | Description                                                                                                                                                          |
| ------------------- | -------------------------------------------------- | ----------------- | --------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| tmax_days_35C       | \# days Tmax ≥35°C                                 | extreme index     | \# days   | Number of days, per year, with Tmax >=35C                                                                                                                            |
| tmax_days_90th      | \# days Tmax ≥90th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 90th percentile. 90th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_95th      | \# days Tmax ≥95th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 95th percentile. 95th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_99th      | \# days Tmax ≥99th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 99th percentile. 99th percentile calculated using all daily tmax values from 1995-2014.                                          |
| Hottest_Tmax        | Hottest Tmax of the Year (°C)                      | extreme index     | degrees C | Hottest Tmax value every year                                                                                                                                        |
| Max_DTR             | Largest Diurnal Temperature Range of the Year (°C) | extreme index     | degrees C | largest diurnal temperature range (tmax minus tmin) each year                                                                                                        |
| tmin_tropnights_20C | \# days Tmin ≥20°C                                 | extreme index     | \# days   | Number of days, per year with Tmin >=20C                                                                                                                             |
| tmin_frostdays_0C   | \# days Tmin ≤0°C                                  | extreme index     | \# days   | Number of days per year with Tmin <=0C                                                                                                                               |
| Coldest_Tmin        | Coldest Tmin of the Year (°C)                      | extreme index     | degrees C | Coldest minimum temperature each year                                                                                                                                |
| prec_days_dry       | \# days with precipitation ≤0.001 in               | extreme index     | \# days   | Number of days, per year, where precipitation <=1e-3 inches                                                                                                          |
| prec_days_oneinch   | \# days with precipitation ≥1 in                   | extreme index     | \# days   | Number of days, per year, where precipitation >=1 inch                                                                                                               |
| prec_days_90th      | \# days with precipitation ≥90th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=90th percentile. 90th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_95th      | \# days with precipitation ≥95th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=95th percentile. 95th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_99th      | \# days with precipitation ≥99th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=99th percentile. 99th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| tmax_annave         | Annual Average Tmax (°C)                           | annual average    | degrees C | Annual average maximum daily temperature                                                                                                                             |
| tmin_annave         | Annual Average Tmin (°C)                           | annual average    | degrees C | Annual average minimum daily temperature                                                                                                                             |
| prec_annave         | Annual Total Precipitation (mm)                    | annual SUM        | degrees C | Annual SUM of precipitation                                                                                                                                          |</details>

In [1]:
# Imports
import os
import warnings
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
import pandasql as psql
import seaborn as sns
from matplotlib import pyplot as plt

# Suppress warnings
warnings.filterwarnings('ignore')

# Configure matplotlib
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['figure.dpi'] = 400
plt.rcParams['font.size'] = 8
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['axes.linewidth'] = 0.1
plt.rcParams['patch.linewidth'] = 0

## Initialization

In [2]:
# Data directory
path = 'updated_extremes'
# NASA center to analysis
center = 'LARC'.upper()
# Flag to use only 2020-2099
only_future = True 

In [3]:
# DO NOT CHANGE THIS CELL
##################################
##          File names          ##
##  variable_CENTER_ssp###.csv  ##
##################################
# NASA Centers
centers = ['AMES', 'GSFC', 'JPL', 'KSC', 'MSFC', 'MAF', 'GISS',
           'LARC', 'SSC', 'GRC', 'WFF', 'JSC', 'WSTF', 'AFRC']

# Temperature variables (number of days)
temp_d = ['tmax_days_35C', 'tmax_days_90th', 'tmax_days_95th', 'tmax_days_99th',
          'tmin_tropnights_20C', 'tmin_frostdays_0C', 'HI_days_100F']

# Temperature variables (degrees Celsius)
temp_c = ['Coldest_Tmin', 'Hottest_Tmax', 'Max_DTR']

# Precipitation variables (number of days)
prec_d = ['prec_days_dry', 'prec_days_oneinch', 'prec_days_90th',
          'prec_days_95th', 'prec_days_99th']

# All variables
variables = sorted(temp_d + temp_c + prec_d)

# Not present in folder
# annave = ['tmax_annave', 'tmin_annave', 'prec_annave']

# Shared Socioeconomic Pathways (SSPs) climate scenarios
ssp = sorted(list({f.split('_')[-1][:-4] for f in os.listdir(path) 
                   if 'ssp' in f.split('_')[-1]}))
print(f'Available: {ssp}')

##################################
##         Column names         ##
##################################

# Climate Models
models = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CMCC-ESM2',
          'CNRM-CM6-1', 'CNRM-ESM2-1', 'EC-Earth3', 'FGOALS-g3', 'GFDL-ESM4',
          'GISS-E2-1-G', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR',
          'KACE-1-0-G', 'MIROC6', 'MIROC-ES2L', 'MPI-ESM1-2-HR',
          'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NorESM2-LM', 'NorESM2-MM']

# Multi-model statistics
mme = ['mme-mean', 'mme-median', 'mme-pct25', 'mme-pct75']

##################################
##         Time Periods         ##
##################################
# 10 years before+after a decade
time_periods = {'short': (2020, 2049),  # 2030's: 2020-2029, 2030-2039, 2040-2049
                'mid':   (2040, 2069),  # 2050's: 2040-2049, 2050-2059, 2060-2069
                'long':  (2070, 2099),  # 2080's: 2070-2079, 2080-2089, 2090-2099
                }

Available: ['ssp126', 'ssp245', 'ssp370', 'ssp585']


# Get Files/Data

In [4]:
def get_files(path: str, center: str, variables: list = None):
    '''
    Returns list of filenames in a directory that contain a specific string.

    Args:
        path: the path to the data directory to search
        center: the NASA center name in the filename
        variables: optional list of variables to filter files (default: None)
    '''
    # Check if the provided center is valid
    if center not in centers:
        raise ValueError(f'{center} not in {centers}')

    # Get all csv files in the directory that contain the center name
    files = [os.path.join(path, f) for f in os.listdir(path) 
             if center in f and f.endswith('.csv')]

    # If provided, filter files that contain any of the variables
    return files if not variables else [f for f in files if any(v in f for v in variables)]
    

# Get files for a center
files = sorted(get_files(path, center))

# Number of files found
print(f'{len(files)} {center} files')
# First 5 files
print(files[:5])

60 LARC files
['updated_extremes/Coldest_Tmin_LARC_ssp126.csv', 'updated_extremes/Coldest_Tmin_LARC_ssp245.csv', 'updated_extremes/Coldest_Tmin_LARC_ssp370.csv', 'updated_extremes/Coldest_Tmin_LARC_ssp585.csv', 'updated_extremes/HI_days_100F_LARC_ssp126.csv']


In [5]:
def check_df_consistency(df_list: list[pd.DataFrame]):
    '''
    Returns T/F if all dataframes in the list have the same shape and columns.
    
    Args:
        df_list: List of pandas DataFrames to check
    '''
    if not df_list:
        return False
    
    # Get reference shape and columns from the first dataframe
    ref_shape, ref_cols = df_list[0].shape, list(df_list[0].columns)
    
    # Check if all other dataframes have the same shape and columns
    return all(df.shape == ref_shape and list(df.columns) == ref_cols for df in df_list[1:])

# Function to label a year with its corresponding term

def label_term(year: int):
    '''
    Returns list of time period labels for the given year
    
    Args:
        year: The year to label
    
    '''
    return [t for t, (s, e) in time_periods.items() if s <= year <= e]


# Function to preprocess a CSV file
def preprocess(filename: str, only_future: bool = True):
    '''
    Returns preprocessed pandas DataFrame based on CSV file
    
    Args:
        filename: Path to the CSV file
        only_future: If True, drop rows with NaN terms (assumed to be past data)
    '''
    df = pd.read_csv(filename)
    
    # Extract variable name from the filename
    name = filename.split('/')[-1][:-4].split('_')
    var = '_'.join(name[:-2])
    
    # Add new columns: term, scenario, and variable
    df.insert(0, 'term', df.years.apply(label_term))
    df.insert(0, 'scenario', name[-1])
    df.insert(0, 'variable', var + ('_c' if var in temp_c else '_d'))
    
    # Explode the 'term' column (in case a year belongs to multiple terms)
    df = df.explode('term')

    # Remove rows with NaN terms if only_future is True, otherwise return all rows
    return df.dropna(subset=['term']) if only_future else df

# Preprocess all files
df = [preprocess(f, only_future) for f in files]

# Check if all dataframes have the same format
if not check_df_consistency(df):
    raise ValueError('DataFrames are inconsistent')

# Combine all dataframes into one
df = pd.concat(df).reset_index(drop=True)

# Check the number of years per time period (expected is 30)
years_per_term = df.groupby(['variable', 'scenario', 'term']).size().unique()
if len(years_per_term) != 1 or years_per_term[0] != 30:
    raise ValueError(
        f'# of years per time period is incorrect: {years_per_term}')


# REMOVE THIS PART IF ERROR FIXED
# For: AMES LARC GISS JPL JSC KSC WFF
# Erronous: Max DTR, Tmin >20C, Tmin < 0C, Coldest Tmin of the Year, Annual Average Tmin, any of the humid heat diagnostics (e.g., heat index, WBGT, etc) in ssp126 and 370
errors = ['Max_DTR', 'tmin', 'Coldest_Tmin']
if center in ['AMES', 'LARC', 'GISS', 'JPL', 'JSC', 'KSC', 'WFF']:
    # Remove rows with erroneous variables for specific scenarios
    df = df[~(df.variable.str.contains('|'.join(errors), na=False, regex=True) &
              df.scenario.isin(['ssp126', 'ssp370']))]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4680 entries, 90 to 5399
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   variable       4680 non-null   object 
 1   scenario       4680 non-null   object 
 2   term           4680 non-null   object 
 3   years          4680 non-null   float64
 4   ACCESS-CM2     4680 non-null   float64
 5   ACCESS-ESM1-5  4680 non-null   float64
 6   BCC-CSM2-MR    4680 non-null   float64
 7   CESM2          2160 non-null   float64
 8   CMCC-ESM2      4680 non-null   float64
 9   CNRM-CM6-1     4680 non-null   float64
 10  CNRM-ESM2-1    4680 non-null   float64
 11  EC-Earth3      4680 non-null   float64
 12  FGOALS-g3      4680 non-null   float64
 13  GFDL-ESM4      4680 non-null   float64
 14  GISS-E2-1-G    4680 non-null   float64
 15  INM-CM4-8      4680 non-null   float64
 16  INM-CM5-0      4680 non-null   float64
 17  IPSL-CM6A-LR   4680 non-null   float64
 18  KACE-1-0-G  

In [6]:
# Calculate average for each term
cols = ['variable', 'scenario', 'term']
# term_mme = df.groupby(cols)[mme].mean().reset_index(
# ).sort_values(cols, ascending=[1, 1, 0], ignore_index=True)

# Calculate term-wise statistics
# For variables ending with '_d', use median
# For variables ending with '_c', use mean
term_mme = pd.concat([df[df.variable.str.endswith('_d')].groupby(cols)[mme].median(),
                      df[df.variable.str.endswith('_c')].groupby(cols)[mme].mean()
                     ]).reset_index().sort_values(cols, ascending=[1, 1, 0],
                                                  ignore_index=True)
display(term_mme.head())

,variable,scenario,term,mme-mean,mme-median,mme-pct25,mme-pct75
0,Coldest_Tmin_c,ssp245,short,-11.268358,-10.740965,-13.503494,-8.679038
1,Coldest_Tmin_c,ssp245,mid,-10.260196,-9.926980,-12.113423,-7.821688
2,Coldest_Tmin_c,ssp245,long,-8.900938,-8.415058,-10.516808,-6.578249
3,Coldest_Tmin_c,ssp585,short,-11.102316,-10.598861,-13.038477,-8.582722
4,Coldest_Tmin_c,ssp585,mid,-9.320134,-9.006964,-11.001367,-7.270380


In [7]:
# Recalculate mme columns
df['calc_mean'] = df[models].mean(axis=1)
df['calc_median'] = df[models].median(axis=1)
df['calc_pct25'] = df[models].quantile(0.25, axis=1)
df['calc_pct75'] = df[models].quantile(0.75, axis=1)
calc = ['calc_mean', 'calc_median', 'calc_pct25', 'calc_pct75']
# term_calc = df.groupby(cols)[calc].mean(
# ).reset_index().sort_values(cols, ascending=[1, 1, 0], ignore_index=True)

# Calculate term-wise statistics
# For variables ending with '_d', use median
# For variables ending with '_c', use mean
term_calc = pd.concat([df[df.variable.str.endswith('_d')].groupby(cols)[calc].median(),
                      df[df.variable.str.endswith('_c')].groupby(cols)[calc].mean()
                     ]).reset_index().sort_values(cols, ascending=[1, 1, 0],
                                                  ignore_index=True)
# Rename columns to match term_mme
term_calc.columns = term_mme.columns

display(term_calc.head())

,variable,scenario,term,mme-mean,mme-median,mme-pct25,mme-pct75
0,Coldest_Tmin_c,ssp245,short,-11.268358,-10.740965,-13.503494,-8.679038
1,Coldest_Tmin_c,ssp245,mid,-10.260196,-9.926980,-12.113423,-7.821688
2,Coldest_Tmin_c,ssp245,long,-8.900938,-8.415058,-10.516808,-6.578249
3,Coldest_Tmin_c,ssp585,short,-11.102316,-10.598861,-13.038477,-8.582722
4,Coldest_Tmin_c,ssp585,mid,-9.320134,-9.006964,-11.001367,-7.270380


In [8]:
# Check if original mme and recalculated values are equal
display((term_mme == term_calc).sum())
cc = term_mme.compare(term_calc, result_names=('term_mme', 'term_calc'))
display(cc)

variable      156
scenario      156
term          156
mme-mean      141
mme-median    156
mme-pct25     156
mme-pct75     156
dtype: int64

mme-mean            
       term_mme   term_calc
15     9.681818    9.681818
48    11.681818   11.681818
50    12.022727   12.022727
54    11.545455   11.545455
55    12.181818   12.181818
59    13.045455   13.045455
70     3.136364    3.136364
105   15.380952   15.380952
106   25.404762   25.404762
119  108.333333  108.333333
133    9.785714    9.785714
134    9.309524    9.309524
151   94.214286   94.214286
153   89.476190   89.476190
154  102.357143  102.357143

## Calculate Change Per (variable, scenario)
- short - mid
- short - long
- mid - long

In [9]:
query = """
SELECT
    a.variable,
    a.scenario,
    CASE
        WHEN a.term = 'short' AND b.term = 'mid' THEN 'short-mid'
        WHEN a.term = 'short' AND b.term = 'long' THEN 'short-long'
        WHEN a.term = 'mid' AND b.term = 'long' THEN 'mid-long'
    END AS term_diff,
    b.'mme-mean' - a.'mme-mean' AS 'mme-mean',
    b.'mme-median' - a.'mme-median' AS 'mme-median',
    b.'mme-pct25' - a.'mme-pct25' AS 'mme-pct25',
    b.'mme-pct75' - a.'mme-pct75' AS 'mme-pct75'
FROM term_mme a
JOIN term_mme b
    ON a.variable = b.variable
    AND a.scenario = b.scenario
    AND (
        (a.term = 'short' AND b.term = 'mid') OR
        (a.term = 'short' AND b.term = 'long') OR
        (a.term = 'mid' AND b.term = 'long')
    )
ORDER BY 1, 2, 3 DESC
"""

change = psql.sqldf(query, locals())

display(change.head())

,variable,scenario,term_diff,mme-mean,mme-median,mme-pct25,mme-pct75
0,Coldest_Tmin_c,ssp245,short-mid,1.008162,0.813985,1.390072,0.857351
1,Coldest_Tmin_c,ssp245,short-long,2.367420,2.325906,2.986686,2.100789
2,Coldest_Tmin_c,ssp245,mid-long,1.359258,1.511921,1.596615,1.243439
3,Coldest_Tmin_c,ssp585,short-mid,1.782182,1.591897,2.037109,1.312342
4,Coldest_Tmin_c,ssp585,short-long,4.379275,4.107285,4.715770,3.710847


# Results

In [10]:
# Create the filter dropdown
filter_dropdown = widgets.SelectMultiple(
    options=['All'] + term_mme.variable.unique().tolist(), 
    value=('All',), 
    description='Filter:'
)

# Create the sort dropdown
sort_dropdown = widgets.Dropdown(
    options=['Default'] + mme,
    value='Default',
    description='Sort by:'
)

def update_display(filter_selection, sort_by):
    def process_dataframe(df):
        if 'All' not in filter_selection:
            df = df[df.variable.isin(filter_selection)]
        
        if sort_by != 'Default':
            df = df.sort_values(by=sort_by)
        
        display(df)
        col = [c for c in df.columns if 'term' in c][0]
        print('Average of Table Above')
        display(df.groupby(col)[mme].mean().reset_index()
            .sort_values(col, ascending=False))
        
    
    print('Short/Mid/Long Term Averages')
    process_dataframe(term_mme)
    print('Short/Mid/Long Term Changes')
    process_dataframe(change)


# Create the interactive widget
interactive_widget = widgets.interactive(
    update_display, 
    filter_selection=filter_dropdown,
    sort_by=sort_dropdown
)

# Display the interactive widget
display(interactive_widget)

interactive(children=(SelectMultiple(description='Filter:', index=(0,), options=('All', 'Coldest_Tmin_c', 'HI_…

## Variables

In [11]:
print('variables:')
display({i: variables[i] for i in range(len(variables))})
print('\nssp scenario:')
display({i: ssp[i] for i in range(len(ssp))})

variables:


{0: 'Coldest_Tmin',
 1: 'HI_days_100F',
 2: 'Hottest_Tmax',
 3: 'Max_DTR',
 4: 'prec_days_90th',
 5: 'prec_days_95th',
 6: 'prec_days_99th',
 7: 'prec_days_dry',
 8: 'prec_days_oneinch',
 9: 'tmax_days_35C',
 10: 'tmax_days_90th',
 11: 'tmax_days_95th',
 12: 'tmax_days_99th',
 13: 'tmin_frostdays_0C',
 14: 'tmin_tropnights_20C'}


ssp scenario:


{0: 'ssp126', 1: 'ssp245', 2: 'ssp370', 3: 'ssp585'}

## Short/Mid/Long Term Averages/Median

In [12]:
# Change number/index in square brackets 
# to get variable/ssp name according to ouput above
var = variables[2]
s = ssp[0]

print(f'Results for {var} and maybe {s}')
print('Original Table')
temp = term_mme[term_mme.variable.str.contains(var)]#[term_mme.scenario==s]
display(temp)

print('Overall Statistics')
display(temp.describe())

# Group by 'term' and apply aggregation
print('Short/Mid/Long Term Statistics')
display(temp[['term'] + mme].groupby('term')
        .agg({col: ['min', 'median', 'max'] for col in mme}))

Results for Hottest_Tmax and maybe ssp126
Original Table


,variable,scenario,term,mme-mean,mme-median,mme-pct25,mme-pct75
18,Hottest_Tmax_c,ssp126,short,39.446548,39.302368,37.945853,40.872249
19,Hottest_Tmax_c,ssp126,mid,39.947213,39.703359,38.368416,41.430420
20,Hottest_Tmax_c,ssp126,long,39.878945,39.653520,38.303831,41.313862
21,Hottest_Tmax_c,ssp245,short,39.078562,38.982095,37.727870,40.399634
22,Hottest_Tmax_c,ssp245,mid,39.553230,39.477477,38.136423,40.921464
23,Hottest_Tmax_c,ssp245,long,40.112750,40.004212,38.667057,41.446354
24,Hottest_Tmax_c,ssp370,short,39.775092,39.792292,38.341625,41.182024
25,Hottest_Tmax_c,ssp370,mid,40.452730,40.523494,38.920641,41.810544
26,Hottest_Tmax_c,ssp370,long,41.974741,41.833870,40.486604,43.474324
27,Hottest_Tmax_c,ssp585,short,38.901492,38.733753,37.511954,40.175730


Overall Statistics


,mme-mean,mme-median,mme-pct25,mme-pct75
count,12.000000,12.000000,12.000000,12.000000
mean,40.096397,39.981779,38.625648,41.486884
std,1.005259,1.015422,0.950488,1.049468
min,38.901492,38.733753,37.511954,40.175730
25%,39.526560,39.433700,38.088780,40.909160
50%,39.899589,39.738270,38.355021,41.286780
75%,40.197745,40.134033,38.730453,41.537402
max,42.115223,42.001719,40.486604,43.556306


Short/Mid/Long Term Statistics


mme-mean                       mme-median                        \
             min     median        max        min     median        max   
term                                                                      
long   39.878945  41.043746  42.115223  39.653520  40.919041  42.001719   
mid    39.553230  39.933723  40.452730  39.477477  39.738270  40.523494   
short  38.901492  39.262555  39.775092  38.733753  39.142232  39.792292   

       mme-pct25                        mme-pct75                        
             min     median        max        min     median        max  
term                                                                     
long   38.303831  39.566624  40.486604  41.313862  42.460339  43.556306  
mid    38.136423  38.499862  38.920641  40.921464  41.345059  41.810544  
short  37.511954  37.836861  38.341625  40.175730  40.635942  41.182024

## Short/Mid/Long Term Averages/Median Changes

In [13]:
# Change number/index in square brackets 
# to get variable/ssp name according to ouput above
var = variables[0]
s = ssp[0]

print(f'Results for {var} and maybe {s}')
print('Change Table')
temp = change[change.variable.str.contains(var)]#[term_mme.scenario==s]
display(temp)

print('Overall Statistics')
display(temp.describe())

# Group by 'term' and apply aggregation
print('Short/Mid/Long Term Statistics')
display(temp[['term_diff'] + mme].groupby('term_diff')
        .agg({col: ['min', 'median', 'max'] for col in mme}))

Results for Coldest_Tmin and maybe ssp126
Change Table


,variable,scenario,term_diff,mme-mean,mme-median,mme-pct25,mme-pct75
0,Coldest_Tmin_c,ssp245,short-mid,1.008162,0.813985,1.390072,0.857351
1,Coldest_Tmin_c,ssp245,short-long,2.367420,2.325906,2.986686,2.100789
2,Coldest_Tmin_c,ssp245,mid-long,1.359258,1.511921,1.596615,1.243439
3,Coldest_Tmin_c,ssp585,short-mid,1.782182,1.591897,2.037109,1.312342
4,Coldest_Tmin_c,ssp585,short-long,4.379275,4.107285,4.715770,3.710847
5,Coldest_Tmin_c,ssp585,mid-long,2.597093,2.515388,2.678661,2.398505


Overall Statistics


,mme-mean,mme-median,mme-pct25,mme-pct75
count,6.000000,6.000000,6.000000,6.000000
mean,2.248898,2.144397,2.567486,1.937212
std,1.201874,1.140069,1.217654,1.041462
min,1.008162,0.813985,1.390072,0.857351
25%,1.464989,1.531915,1.706738,1.260665
50%,2.074801,1.958901,2.357885,1.706566
75%,2.539675,2.468018,2.909680,2.324076
max,4.379275,4.107285,4.715770,3.710847


Short/Mid/Long Term Statistics


mme-mean                     mme-median                      \
                 min    median       max        min    median       max   
term_diff                                                                 
mid-long    1.359258  1.978176  2.597093   1.511921  2.013655  2.515388   
short-long  2.367420  3.373348  4.379275   2.325906  3.216595  4.107285   
short-mid   1.008162  1.395172  1.782182   0.813985  1.202941  1.591897   

           mme-pct25                     mme-pct75                      
                 min    median       max       min    median       max  
term_diff                                                               
mid-long    1.596615  2.137638  2.678661  1.243439  1.820972  2.398505  
short-long  2.986686  3.851228  4.715770  2.100789  2.905818  3.710847  
short-mid   1.390072  1.713590  2.037109  0.857351  1.084846  1.312342

# Quality check Plots - uncomment to run

In [14]:
# unit_filter = ['_c', '_d'][1]

# dropdown = widgets.SelectMultiple(
#     options=['All'] +(term_mme.variable[term_mme.variable.str.endswith(unit_filter)]
#                       .unique().tolist()),
#     value=('All',), description='Filter:'
# )

# def update_plot(selection):
#     filter_condition = (term_mme.variable.str.endswith(unit_filter) 
#                         if 'All' in selection 
#                         else term_mme.variable.isin(selection))
#     term_mme_f, change_f = term_mme[filter_condition], change[filter_condition]
    
#     unit_display = '# days' if unit_filter == '_d' else 'degree C'
    
#     for data, title in [(term_mme_f, 'Term MME'), (change_f, 'Change')]:
#         pp = sns.pairplot(data=data, vars=mme, hue='variable', 
#                           corner=True, plot_kws={'alpha': 0.5})
#         pp.fig.suptitle(f'{title}: unit({unit_display})')
    
#     plt.show()

# widgets.interactive(update_plot, selection=dropdown)


## Time Series: check for sudden spikes/drops

In [15]:
# temp = df[df.variable.str.contains(temp_d[0])][df.scenario==ssp[0]]

# for m in models:
#     sns.lineplot(temp, x='years', y=m, label=m, linewidth=0.1)
# plt.show()

# for m in mme:
#     sns.lineplot(temp, x='years', y=m, label=m, linewidth=0.2)
# plt.show()